In [ ]:
!pip install -q mediapipe

In [ ]:
pip install tensorflow

In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
actions=np.array(['hello','thanks','iloveyou','victory','father','mother','help','yes','no'])
no_sequences=30
sequence_length=30

In [ ]:
label_map = {label:num for num, label in enumerate(actions)}

In [ ]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [ ]:
X = np.array(sequences) 

In [ ]:
y = to_categorical(labels).astype(int)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [ ]:
y_test.shape

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from keras.callbacks import EarlyStopping
from keras.layers import Dropout

In [ ]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
from keras.layers import Dense, Dropout, Flatten

In [ ]:
model = Sequential()
model.add(Flatten(input_shape=(30,126)))
model.add(Dense(64,  activation='relu'))
model.add(Dropout(0.2)) # add dropout layer
model.add(Dense(128,  activation='relu'))
model.add(Dropout(0.2)) # add dropout layer
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2)) # add dropout layer
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2)) # add dropout layer
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2)) # add dropout layer
model.add(Dense(9, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, min_delta=0.001, mode='min', verbose=1)
model.fit(X_train,y_train,epochs=500, batch_size=32,callbacks=[early_stopping],validation_data=(X_test, y_test))

In [ ]:
model.summary()

In [ ]:
model.save('actionmodel.h5') 

In [ ]:
model.load_weights('actionmodel.h5')

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)